In [611]:
from spotipy import Spotify
import spotipy
import spotipy.util as util
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame as df
import pprint
import json
pp = pprint.PrettyPrinter(indent=2)
final_df = pd.DataFrame()

In [612]:
if __name__ == "__main__":
    user = 'iamthatbenj'
    scope = 'user-read-recently-played'
    token = util.prompt_for_user_token(user, scope, 'dd16ba33195e4591b57889231485ceec', '6148f018d44c4683a9b37b6cbe027142', 'http://localhost:8888/callback/')
    sp = spotipy.Spotify(auth=token)
    
    if token:
        recent = sp.current_user_recently_played(limit=50)

In [613]:
song_counter = 0
title = []
artist = []
artist_id = []
playtime = []
song_id = []

while song_counter < 50:
    title.append(recent['items'][song_counter]['track']['name'])
    artist.append(recent['items'][song_counter]['track']['artists'][0]['name'])
    artist_id.append(recent['items'][song_counter]['track']['artists'][0]['id'])
    playtime.append(recent['items'][song_counter]['played_at'])
    song_id.append(recent['items'][song_counter]['track']['id'])
    song_counter += 1

title_df = pd.DataFrame(title)
artist_df = pd.DataFrame(artist)
artist_id_df = pd.DataFrame(artist_id)
playtime_df = pd.DataFrame(playtime)
song_id_df = pd.DataFrame(song_id)

transfer = [title_df, artist_df, artist_id_df, playtime_df, song_id_df]
playback_raw = pd.concat(transfer, axis=1, sort=False)
playback_raw.columns = ['title','artist','artist_id','playtime','song_id']
playback_raw['playtime'] = pd.to_datetime(playback_raw['playtime'])
playback_raw['playtime'] = playback_raw['playtime'].dt.round('1s')
playback_new = playback_raw.drop_duplicates(subset='playtime', keep='first')

In [614]:
query_songs = list(playback_new.song_id)

In [615]:
audio_df = pd.DataFrame()

if token:
    for i in query_songs:
        audio = sp.audio_features(i)
        audio_df = audio_df.append(audio)
        
audio_df.rename(columns={'id' : 'song_id'}, inplace=True)

In [617]:
playback_ready = playback_new.merge(audio_df, on='song_id')

In [618]:
merge_df = pd.DataFrame()
artist_input = list(playback_ready['artist_id'])

In [619]:
art_query = sp.artists(artist_input)
aq = 0

artist_id = []
genres = []
popularity = []

while aq < len(artist_input):
    genres.append(art_query['artists'][aq]['genres'])
    artist_id.append(art_query['artists'][aq]['id'])
    popularity.append(art_query['artists'][aq]['popularity'])
    aq += 1
    
artist_id_df = pd.DataFrame(artist_id)
genre_df = pd.Series(genres)
popularity_df = pd.DataFrame(popularity)
transfer2 = [artist_id_df, genre_df, popularity_df]
artist_raw = pd.concat(transfer2, axis=1, sort=False)
artist_raw.columns = ('artist_id', 'genres', 'popularity')

In [622]:
merge_df = playback_ready.merge(artist_raw)
final_df = merge_df.drop_duplicates('playtime', keep='first')

In [623]:
final_df.head(50)

,title,artist,artist_id,playtime,song_id,acousticness,analysis_url,danceability,duration_ms,energy,...,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,genres,popularity
0,Hold You Now (feat. Danielle Haim),Vampire Weekend,5BvJzeQpmsdsFp4HGUYUEx,2019-05-06 22:30:00,15E2P2ToSLNnZvcOEQra6a,0.081700,https://api.spotify.com/v1/audio-analysis/15E2...,0.284,153787,0.4570,...,1,0.0382,86.880,4,https://api.spotify.com/v1/tracks/15E2P2ToSLNn...,audio_features,spotify:track:15E2P2ToSLNnZvcOEQra6a,0.2930,"[chamber pop, folk-pop, indie folk, indie pop,...",78
1,Cattails,Big Thief,5QdyldG4Fl4TPiOIeMNpBZ,2019-05-06 22:27:27,623gFZ4VYa2EDu1D1sWFm0,0.822000,https://api.spotify.com/v1/audio-analysis/623g...,0.556,245640,0.5800,...,1,0.0249,80.514,4,https://api.spotify.com/v1/tracks/623gFZ4VYa2E...,audio_features,spotify:track:623gFZ4VYa2EDu1D1sWFm0,0.7020,"[art pop, brooklyn indie, chamber pop, dream p...",64
4,UFOF,Big Thief,5QdyldG4Fl4TPiOIeMNpBZ,2019-05-06 22:26:36,1DXvEWgI1xoxmKv2Zd3pNE,0.638000,https://api.spotify.com/v1/audio-analysis/1DXv...,0.623,188413,0.6130,...,0,0.0309,147.168,4,https://api.spotify.com/v1/tracks/1DXvEWgI1xox...,audio_features,spotify:track:1DXvEWgI1xoxmKv2Zd3pNE,0.7450,"[art pop, brooklyn indie, chamber pop, dream p...",64
7,Contact,Big Thief,5QdyldG4Fl4TPiOIeMNpBZ,2019-05-06 22:23:28,4ByIVtEsADLpBFcI8oY4hC,0.895000,https://api.spotify.com/v1/audio-analysis/4ByI...,0.453,234493,0.3830,...,0,0.0245,79.254,4,https://api.spotify.com/v1/tracks/4ByIVtEsADLp...,audio_features,spotify:track:4ByIVtEsADLpBFcI8oY4hC,0.3110,"[art pop, brooklyn indie, chamber pop, dream p...",64
10,Fine,The Modern Jazz Quartet,7wBFjZMHsC6nfV0HOSd6uI,2019-05-06 21:54:12,7ApvA0RUmTBuvQVHJuYkNi,0.929000,https://api.spotify.com/v1/audio-analysis/7Apv...,0.506,369520,0.0394,...,0,0.0454,152.780,4,https://api.spotify.com/v1/tracks/7ApvA0RUmTBu...,audio_features,spotify:track:7ApvA0RUmTBuvQVHJuYkNi,0.3220,"[bebop, big band, contemporary post-bop, cool ...",48
12,DA Capo,The Modern Jazz Quartet,7wBFjZMHsC6nfV0HOSd6uI,2019-05-06 21:47:03,1R0bvt350ZxB8BsMUtKrAO,0.948000,https://api.spotify.com/v1/audio-analysis/1R0b...,0.453,221053,0.0625,...,1,0.0406,113.472,4,https://api.spotify.com/v1/tracks/1R0bvt350ZxB...,audio_features,spotify:track:1R0bvt350ZxB8BsMUtKrAO,0.1870,"[bebop, big band, contemporary post-bop, cool ...",48
14,Subterranean Homesick Alien,Radiohead,4Z8W4fKeB5YxbusRsdQVPb,2019-05-06 21:39:20,19yGmm9FjEZdZc5j98WDe4,0.049800,https://api.spotify.com/v1/audio-analysis/19yG...,0.316,267693,0.5920,...,1,0.0296,150.537,3,https://api.spotify.com/v1/tracks/19yGmm9FjEZd...,audio_features,spotify:track:19yGmm9FjEZdZc5j98WDe4,0.3200,"[alternative rock, art rock, melancholia, mode...",80
17,Paranoid Android,Radiohead,4Z8W4fKeB5YxbusRsdQVPb,2019-05-06 21:36:34,2nTsKOXIVGDf2iPeVQO2Gm,0.037700,https://api.spotify.com/v1/audio-analysis/2nTs...,0.252,387213,0.8490,...,1,0.0581,163.704,4,https://api.spotify.com/v1/tracks/2nTsKOXIVGDf...,audio_features,spotify:track:2nTsKOXIVGDf2iPeVQO2Gm,0.1940,"[alternative rock, art rock, melancholia, mode...",80
20,Airbag,Radiohead,4Z8W4fKeB5YxbusRsdQVPb,2019-05-06 21:30:07,3OsUjkcv1C1v5udFhgRSFg,0.007920,https://api.spotify.com/v1/audio-analysis/3OsU...,0.306,287880,0.8720,...,1,0.0500,167.946,4,https://api.spotify.com/v1/tracks/3OsUjkcv1C1v...,audio_features,spotify:track:3OsUjkcv1C1v5udFhgRSFg,0.6480,"[alternative rock, art rock, melancholia, mode...",80
23,Cousin Mary,John Coltrane,2hGh5VOeeqimQFxqXvfCUf,2019-05-06 21:25:08,0DBEfSktiApw6NoTpH1Sjc,0.473000,https://api.spotify.com/v1/audio-analysis/0DBE...,0.529,349533,0.3540,...,1,0.0546,111.434,4,https://api.spotify.com/v1/tracks/0DBEfSktiApw...,audio_features,spotify:track:0DBEfSktiApw6NoTpH1Sjc,0.4430,"[bebop, contemporary post-bop, cool jazz, free...",66
